<a href="https://colab.research.google.com/github/CristianCosci/AGV-Project/blob/refactoring_mobilenet/AGV_emotions_attack/test_script/prove_AGV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -b refactoring_mobilenet https://github.com/CristianCosci/AGV-Project.git

In [ ]:
!pip install noise

In [ ]:
import sys, os
os.chdir('/content/AGV-Project/AGV_emotions_attack/attacks_emotions/agv/')

import random
import argparse
#set modules path
sys.path.append(os.path.dirname('attack.py'))
sys.path.append(os.path.join(os.path.dirname('attack.py'),'..'))
sys.path.append(os.path.join(os.path.dirname('attack.py'),'..', ".."))
import numpy as np
import pandas as pd

from fitness import inv_attack_rate, inv_attack_rate_multiple
from agv_filters import Filters
from agv_model_loader import ModelLoader
from agv_optimizer import AGVOptimizer
from agv_optimizer import Individual
from agv_datasets import build_model_and_dataset
from agv_datasets import get_model_name_from_dataset
from agv_datasets import database_and_model
from agv_distances import get_distance_functions
from agv_metrics import compute_metricts

from agv_tests import test, test_fits, save_adv_ex  
from agv_tests import mkdir_p, save_adv_best
from log import Log

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils

model, _, _ = build_model_and_dataset('IMAGENET-MOBILENET')

In [ ]:
os.chdir('/content/AGV-Project/AGV_emotions_attack/')
file_list = os.listdir('attacks_emotions/datasets/images_dataset/ILSVRC2012_img_val_labeled_caffe_200')
origin_class = []
for name in file_list:
  splitted_name = name.split('.')
  origin_class.append(splitted_name[0])

origin_class = [int(i) for i in origin_class]
print(origin_class)

img_path = 'attacks_emotions/datasets/images_dataset/ILSVRC2012_img_val_labeled_caffe_200/'
images = []
for file in file_list:
  img = image.load_img(img_path + file, target_size=(224, 224))
  img_array = image.img_to_array(img)/255
  img_array_expanded_dims = np.expand_dims(img_array, axis=0)
  images.append(img_array_expanded_dims)

images = np.vstack(images)

In [10]:
predictions = model.predict(images, batch_size=10)
result = imagenet_utils.decode_predictions(predictions)
pred_class = [np.argmax(x) for x in predictions]

In [24]:
# count = 0
for i in range(200):
  if pred_class[i] != origin_class[i]:
    print('ELEMENTO: {}, classe originale: {} --- classe predetta: {}'.format(i, origin_class[i], pred_class[i]))

ELEMENTO: 1, classe originale: 493 --- classe predetta: 894
ELEMENTO: 6, classe originale: 198 --- classe predetta: 196
ELEMENTO: 7, classe originale: 343 --- classe predetta: 360
ELEMENTO: 9, classe originale: 516 --- classe predetta: 431
ELEMENTO: 12, classe originale: 620 --- classe predetta: 681
ELEMENTO: 15, classe originale: 841 --- classe predetta: 903
ELEMENTO: 18, classe originale: 479 --- classe predetta: 436
ELEMENTO: 19, classe originale: 57 --- classe predetta: 65
ELEMENTO: 21, classe originale: 486 --- classe predetta: 889
ELEMENTO: 30, classe originale: 703 --- classe predetta: 637
ELEMENTO: 33, classe originale: 725 --- classe predetta: 505
ELEMENTO: 36, classe originale: 970 --- classe predetta: 795
ELEMENTO: 42, classe originale: 230 --- classe predetta: 231
ELEMENTO: 43, classe originale: 975 --- classe predetta: 977
ELEMENTO: 45, classe originale: 878 --- classe predetta: 810
ELEMENTO: 47, classe originale: 28 --- classe predetta: 119
ELEMENTO: 50, classe originale:

In [ ]:
from IPython.display import Image
os.chdir('/content/AGV-Project/AGV_emotions_attack/')
Image(filename=img_path+'65.1.JPEG')

In [48]:
img = image.load_img(img_path + '65.1.JPEG', target_size=(224, 224))
img_array = image.img_to_array(img)/255
img_array_expanded_dims = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array_expanded_dims)
result = imagenet_utils.decode_predictions(predictions)
print(np.argmax((predictions)))

65


In [ ]:
!python attacks_emotions/agv/agv_attack.py -l TEST.txt -o TEST.json -bs 1 -e 10 -pp  "offsprings" -ps "direct" -po "ES"  -np 10 -el true -s pareto -df1 ssim -db "IMAGENET-MOBILENET"  -nf 3 -r true -lf TEST.out

In [ ]:
!python attacks_emotions/agv/agv_attack.py -l TEST.txt -o TEST.json -bs 1 -e 1 -pp  "offsprings" -ps "direct" -po "ES"  -np 1 -el true -s pareto -df1 ssim -db "IMAGENET-MOBILENET"  -nf 3 -r true -lf TEST.out -sae 1

In [ ]:
!python attacks_emotions/agv/agv_attack.py -bf TEST/best_jsons -db "IMAGENET-MOBILENET" -sae_best=True -img_id 10